<a href="https://colab.research.google.com/github/devyulbae/AIClass/blob/main/2)_LLMs_novel_chain_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 패키지 설치

In [2]:
!pip install openai langchain langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all th

In [4]:
from pprint import pprint
from typing import Dict, List

from langchain.chains import LLMChain, SequentialChain
from langchain.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.chat import ChatPromptTemplate
from pydantic import BaseModel


### OpenAI API key

In [12]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


### Prompt chain 준비
* 서비스할 내용의 프롬프트 체인을 준비합니다.
* 각 프롬프트 체인을 미리 준비해 놓고, 템플릿으로 사용합니다.

In [14]:
P_PATH = "/content/drive/MyDrive/datas/multi_prompt"
IDEA_P = os.path.join(P_PATH, "extract_idea.txt")
OUTLINE_P = os.path.join(P_PATH, "write_outline.txt")
PLOT_P = os.path.join(P_PATH, "write_plot.txt")
CHAPTER_P = os.path.join(P_PATH, "write_chapter.txt")

### Prompt chain 구현
* `SequentialChain`을 이용해서 여러개의 chain을 연속적으로 구현할 수 있습니다.

In [15]:
class UserRequest(BaseModel):
    genre: str
    characters: List[Dict[str, str]]
    text: str


def read_prompt_template(file_path: str) -> str:
    with open(file_path, "r") as f:
        prompt_template = f.read()

    return prompt_template


def create_chain(llm, template_path, output_key):
    return LLMChain(
        llm=llm,
        prompt=ChatPromptTemplate.from_template(
            template=read_prompt_template(template_path),
        ),
        output_key=output_key,
        verbose=True,
    )


def generate_novel(req: UserRequest) -> Dict[str, str]:
    writer_llm = ChatGoogleGenerativeAI(model="gemini-pro")
    # ChatOpenAI(temperature=0.3, max_tokens=500, model="gpt-3.5-turbo")

    # 아이디어 뽑기 체인 생성
    novel_idea_chain = create_chain(writer_llm, IDEA_P, "novel_idea")

    # 아웃라인 작성 체인 생성
    novel_outline_chain = create_chain(
        writer_llm, OUTLINE_P, "novel_outline"
    )

    # 플롯 작성 체인 생성
    novel_plot_chain = create_chain(writer_llm, PLOT_P, "novel_plot")

    # 챕터 작성 체인 생성
    novel_chapter_chain = create_chain(writer_llm, CHAPTER_P, "output")

    preprocess_chain = SequentialChain(
        chains=[
            novel_idea_chain,
            novel_outline_chain,
            novel_plot_chain
        ],
        input_variables=["genre", "characters", "text"],
        output_variables=["novel_idea", "novel_outline", "novel_plot"],
        verbose=True,
    )

    context = req.dict()
    context = preprocess_chain(context)

    context["novel_chapter"] = []
    for chapter_number in range(1, 3):
        context["chapter_number"] = chapter_number
        context = novel_chapter_chain(context)
        context["novel_chapter"].append(context["output"])

    contents = "\n\n".join(context["novel_chapter"])
    return {"results": contents}

### User prompt 작성
* User가 직접 작성하는 프롬프트를 작성합니다.

In [22]:
user_data = {
    "genre": "디즈니 판타지",
    "characters": [
        {
            "name": "아샤",
            "role": "마을 소녀이자 주인공"
        },
        {
            "name": "매그니피코",
            "role": "왕국 국왕이자 악당"
        }
    ],
    "text": "So I made this wish."
}


* User Prompt를 입력합니다.

In [23]:
request_instance = UserRequest(**user_data)

### Text Generation

In [24]:
generate_novel(request_instance)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '아샤', 'role': '마을 소녀이자 주인공'}, {'name': '매그니피코', 'role': '왕국 국왕이자 악당'}]

[참고 텍스트]
So I made this wish.

[등장 인물] 과 [참고 텍스트] 를 소재로 새롭고 흥미진진한 디즈니 판타지 소설 아이디어를 한 문단으로 작성해줘

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Human: [등장 인물]
[{'name': '아샤', 'role': '마을 소녀이자 주인공'}, {'name': '매그니피코', 'role': '왕국 국왕이자 악당'}]

[참고 텍스트]
So I made this wish.

[아이디어]
아샤라는 마을 소녀는 세상에 대한 열망이 가득하고 그것을 되돌리려는 욕구로 가득 찬 자비로운 영혼이다. 그녀는 숲에서 매그니피코 왕을 만나고 그가 그녀의 세상에 재앙을 퍼뜨릴 것을 깨닫는다. 아샤는 왕국의 미래를 위해 매그니피코를 저지하기 위해 여행을 떠나고 길을 따라 새로운 친구들을 사귀고, 용감함과 연민의 중요성에 대해 배운다. 그러나 매그니피코는 강력하고 그의 부하들은 무자비하다. 아샤는 용기와 신념을 모아 우정의 힘과 사랑의 힘으로 악의 군대를 물리치기 위해 싸워야만 한다.

[context]
아웃라인 단계에서는 주요 이벤트와 결말을 고려하세요. 
여기서 중요한 것은, 이 단계에서 구체적인 디테일에 매몰되기보다는 스토리의 큰 그림에 집중하는 것입니다.

[등장 인물] 과 [아이디어] 를 소재로 새롭고 흥미진진한 디즈니 판타지 소설의 아웃라인을 작성해줘


> Finished chain.


> Entering new LLMChain chain...
Prompt 

{'results': '아샤의 운명\n\n햇빛을 받으며 기쁨으로 노래하는 아샤는 숲속에서 행복하게 산다. 그녀는 동물들과 함께 어울리고, 그들과 노래를 부르며 즐거운 시간을 보낸다.\n\n"아샤야, 이리와"\n\n아샤의 노래를 들은 리암이 그녀를 부른다. 그는 용감한 전사이며, 아샤를 보호하기 위해 항상 곁에 있다.\n\n"무슨 일이야, 리암?"\n\n"매그니피코가 나타났다. 그는 우리 마을을 파괴하려고 한다."\n\n아샤는 놀란다. 매그니피코는 사악한 왕으로, 그의 마법으로 모든 것을 파괴한다.\n\n"우리 마을을 지켜야 해."\n\n아샤는 리암과 함께 마을로 돌아온다. 마을 사람들은 모두 매그니피코의 부하들에 의해 쫓기고 있다.\n\n"도망쳐! 빨리 도망쳐!"\n\n아샤와 리암은 마을 사람들을 보호하며 싸운다. 그러나 매그니피코의 부하들은 너무 강하다.\n\n"우리는 도망쳐야 해."\n\n아샤와 리암은 마을 사람들을 데리고 숲속 깊은 곳으로 도망친다. 그들은 매그니피코의 부하들로부터 숨어라면서, 그를 물리칠 방법을 찾아야 한다.\n\n아샤는 매그니피코를 물리칠 유일한 방법은 그의 마법의 원천인 마법의 보석을 파괴하는 것이라고 말한다.\n\n"우리는 마법의 보석을 찾아야 해."\n\n아샤, 리암, 그리고 메를린은 마법의 보석을 찾기 위해 여행을 떠난다. 그들은 위험한 숲과 높은 산을 넘어야 한다.\n\n아샤는 용감함과 연민을 배우며, 진정한 영웅으로 성장한다. 그들은 마침내 매그니피코의 성에 도착하고, 그를 물리칠 준비를 한다.\n\n"우리는 꼭 이길 수 있어."\n\n아샤는 자신과 친구들을 격려한다. 그들은 매그니피코와 마지막 대결을 위해 성으로 들어간다.\n\n용감함과 연민의 여정\n\n아샤, 리암, 그리고 메를린은 마법의 보석을 찾기 위해 여행을 떠났다. 그들은 험난한 산과 불길한 숲을 헤치며 여행을 계속했다. 여행을 하며 그들은 매그니피코의 부하들과 싸워야 했다. 매그니피코의 부하들은 강력하고 사악했지만, 아샤와 그녀의 친구들은 용감하게 맞섰다.\n\